In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from pandas import Series, DataFrame
from datetime import datetime
%matplotlib inline

In [2]:
weather=pd.read_csv('austin_weather.csv').drop(['DewPointHighF','DewPointLowF','HumidityHighPercent','HumidityLowPercent',
                                               'SeaLevelPressureHighInches','SeaLevelPressureLowInches','VisibilityHighMiles',
                                               'VisibilityLowMiles','WindHighMPH','WindGustMPH','HumidityAvgPercent'],1)
trips=pd.read_csv('austin_bikeshare_trips.csv')
station=pd.read_csv('austin_bikeshare_stations_w_zipcode.csv').drop(['latitude','longitude','status','name','Unnamed: 0'],1)

In [3]:
weather['Rain']=np.where(weather.Events.str.contains('Rain'), 1, 0)
weather.Date=pd.to_datetime(weather.Date)
weather=weather.set_index('Date').drop('Events',1)

In [4]:
ind = pd.DatetimeIndex(trips.start_time)
trips['Date'] = ind.date.astype('datetime64')
trips['hour'] = ind.hour
by_date = trips.pivot_table('trip_id', aggfunc='count',index='Date',columns='subscriber_type', )
by_date = trips.groupby(['Date', 'subscriber_type'])['trip_id'].count()
by_date.name = 'count'
by_date = by_date.reset_index('subscriber_type')
trips_2=trips[trips.year.isin([2014,2015,2016])]
trip_weather=trips_2.merge(weather,left_on='Date',right_on='Date',
                         suffixes=['_trips','_weather']).drop(['SeaLevelPressureAvgInches'],1)

In [5]:
trip_weather["start_time"] = trip_weather["start_time"].map(lambda x : datetime.strptime(x, "%Y/%m/%d %H:%M"))
trip_weather["hours"] = trip_weather["start_time"].map(lambda x : x.hour)
val=''
def time_of_day(x):
    global val
    if (x>=6)&(x<10):
        val='morning rush hours'
    elif (x>=15)&(x<19):
        val='afternoon ruch hours'
    elif (x>=10)&(x<15):
        val='noon'
    elif (x>=19)|(x<6):
        val='night'
    return val
trip_weather['Time of the day'] = trip_weather['hours'].apply(lambda a: time_of_day(a))

In [6]:
trip_zipcode=trip_weather.merge(station,left_on='start_station_id',right_on='station_id')

In [7]:
trip_zipcode.loc[trip_zipcode['zipcode'] != 78705, 'University_vicinity'] = 0
trip_zipcode.loc[trip_zipcode['zipcode'] == 78705, 'University_vicinity'] = 1
trip_zipcode.head()

,bikeid,checkout_time,duration_minutes,end_station_id,end_station_name,month,start_station_id,start_station_name,start_time,subscriber_type,...,VisibilityAvgMiles,WindAvgMPH,PrecipitationSumInches,Rain,hours,Time of the day,location,station_id,zipcode,University_vicinity
0,8.0,19:12:00,41,2565.0,Trinity & 6th Street,3.0,2536.0,Waller & 6th St.,2015-03-19 19:12:00,Walk Up,...,8,5,0,0,19,night,(30.26461 -97.73049),2536,78702,0.0
1,371.0,19:12:00,6,2822.0,East 6th at Robert Martinez,3.0,2536.0,Waller & 6th St.,2015-03-19 19:12:00,Walk Up,...,8,5,0,0,19,night,(30.26461 -97.73049),2536,78702,0.0
2,922.0,19:12:00,31,2495.0,4th & Congress,3.0,2536.0,Waller & 6th St.,2015-03-19 19:12:00,Walk Up,...,8,5,0,0,19,night,(30.26461 -97.73049),2536,78702,0.0
3,991.0,21:12:00,25,2544.0,East 6th & Pedernales St.,3.0,2536.0,Waller & 6th St.,2015-03-19 21:12:00,Walk Up,...,8,5,0,0,21,night,(30.26461 -97.73049),2536,78702,0.0
4,822.0,21:12:00,0,2536.0,Waller & 6th St.,3.0,2536.0,Waller & 6th St.,2015-03-19 21:12:00,Walk Up,...,8,5,0,0,21,night,(30.26461 -97.73049),2536,78702,0.0


In [8]:
holidays = ['2014-01-01', '2015-01-01', '2016-01-01','2014-01-18', '2014-01-19','2014-01-20','2015-01-17', 
            '2015-01-18','2015-01-19','2016-01-16', '2016-01-17','2016-01-18','2014-02-15', '2014-02-16', '2014-02-17',
            '2015-02-14', '2015-02-15', '2015-02-16','2016-02-13', '2016-02-14', '2016-02-15',
            '2014-05-24', '2014-05-25', '2014-05-26','2015-05-23', '2015-05-24', '2015-05-25','2016-05-28', '2016-05-29', 
            '2016-05-30','2014-07-04', '2014-07-05', '2014-07-06','2015-07-03', '2015-07-04', '2015-07-05',
            '2016-07-02', '2016-07-03', '2016-07-04','2014-08-30', '2014-08-31', '2014-09-01','2015-09-05', '2015-09-06',
            '2015-09-07','2016-09-03', '2016-09-04', '2016-09-05','2014-10-11', '2014-10-12', '2014-10-13',
            '2015-10-10', '2015-10-11', '2015-10-12','2016-10-08', '2016-10-09', '2016-10-10','2014-11-11', '2015-11-11',
            '2016-11-11','2014-11-27', '2014-11-28', '2014-11-29', '2014-11-30','2015-11-26', '2015-11-27', '2015-11-28',
            '2015-11-29','2016-11-24', '2016-11-25', '2016-11-26', '2016-11-27','2014-12-25', '2014-12-26', '2014-12-27',
            '2014-12-28','2015-12-25', '2015-12-26', '2015-12-27', '2015-12-28','2016-12-24', '2016-12-25', '2016-12-26',
            '2016-12-27','2014-03-07', '2014-03-16','2015-03-13', '2015-03-22', '2016-12-27','2014-03-07', '2014-03-16',
            '2015-03-13', '2015-03-22','2016-03-11', '2016-03-20','2014-10-03', '2014-10-05','2014-10-10', '2014-10-12',
            '2015-10-02', '2015-10-04','2015-10-09', '2015-10-11','2016-09-30', '2016-10-02','2016-10-07', '2016-10-09']
trip_zipcode.loc[-trip_zipcode['Date'].isin(holidays), 'Holiday_flag'] = 0
trip_zipcode.loc[trip_zipcode['Date'].isin(holidays), 'Holiday_flag'] = 1
trip_zipcode.head()

,bikeid,checkout_time,duration_minutes,end_station_id,end_station_name,month,start_station_id,start_station_name,start_time,subscriber_type,...,WindAvgMPH,PrecipitationSumInches,Rain,hours,Time of the day,location,station_id,zipcode,University_vicinity,Holiday_flag
0,8.0,19:12:00,41,2565.0,Trinity & 6th Street,3.0,2536.0,Waller & 6th St.,2015-03-19 19:12:00,Walk Up,...,5,0,0,19,night,(30.26461 -97.73049),2536,78702,0.0,0.0
1,371.0,19:12:00,6,2822.0,East 6th at Robert Martinez,3.0,2536.0,Waller & 6th St.,2015-03-19 19:12:00,Walk Up,...,5,0,0,19,night,(30.26461 -97.73049),2536,78702,0.0,0.0
2,922.0,19:12:00,31,2495.0,4th & Congress,3.0,2536.0,Waller & 6th St.,2015-03-19 19:12:00,Walk Up,...,5,0,0,19,night,(30.26461 -97.73049),2536,78702,0.0,0.0
3,991.0,21:12:00,25,2544.0,East 6th & Pedernales St.,3.0,2536.0,Waller & 6th St.,2015-03-19 21:12:00,Walk Up,...,5,0,0,21,night,(30.26461 -97.73049),2536,78702,0.0,0.0
4,822.0,21:12:00,0,2536.0,Waller & 6th St.,3.0,2536.0,Waller & 6th St.,2015-03-19 21:12:00,Walk Up,...,5,0,0,21,night,(30.26461 -97.73049),2536,78702,0.0,0.0


In [9]:
trip_zipcode['weekday'] = trip_zipcode['Date'].apply(lambda x: x.weekday())
trip_zipcode.loc[trip_zipcode['weekday'].isin([1,2,3,4,5]), 'Weekday_flag'] = 1
trip_zipcode.loc[trip_zipcode['weekday'].isin([6,7]), 'Weekday_flag'] = 0
trip_zipcode.head()

,bikeid,checkout_time,duration_minutes,end_station_id,end_station_name,month,start_station_id,start_station_name,start_time,subscriber_type,...,Rain,hours,Time of the day,location,station_id,zipcode,University_vicinity,Holiday_flag,weekday,Weekday_flag
0,8.0,19:12:00,41,2565.0,Trinity & 6th Street,3.0,2536.0,Waller & 6th St.,2015-03-19 19:12:00,Walk Up,...,0,19,night,(30.26461 -97.73049),2536,78702,0.0,0.0,3,1.0
1,371.0,19:12:00,6,2822.0,East 6th at Robert Martinez,3.0,2536.0,Waller & 6th St.,2015-03-19 19:12:00,Walk Up,...,0,19,night,(30.26461 -97.73049),2536,78702,0.0,0.0,3,1.0
2,922.0,19:12:00,31,2495.0,4th & Congress,3.0,2536.0,Waller & 6th St.,2015-03-19 19:12:00,Walk Up,...,0,19,night,(30.26461 -97.73049),2536,78702,0.0,0.0,3,1.0
3,991.0,21:12:00,25,2544.0,East 6th & Pedernales St.,3.0,2536.0,Waller & 6th St.,2015-03-19 21:12:00,Walk Up,...,0,21,night,(30.26461 -97.73049),2536,78702,0.0,0.0,3,1.0
4,822.0,21:12:00,0,2536.0,Waller & 6th St.,3.0,2536.0,Waller & 6th St.,2015-03-19 21:12:00,Walk Up,...,0,21,night,(30.26461 -97.73049),2536,78702,0.0,0.0,3,1.0


In [11]:
trip_zipcode=trip_zipcode.drop(['end_station_id','end_station_name','start_station_name','TempHighF','TempLowF','DewPointAvgF',
                  'VisibilityAvgMiles','hours','location','station_id','PrecipitationSumInches',
                  'start_time','subscriber_type','year','hour','WindAvgMPH'],1)